In [1]:
import numpy as np
import torch

##Per i grafici 3d indagare plotly
from kymatio.torch import HarmonicScattering3D
from kymatio.scattering3d.backend.torch_backend \
    import TorchBackend3D

# Reads Quijote halo catalogues
import readfof

import sys
sys.path.insert(1, '/home/fuffolo97/TESI/MyFunc')
from CalcWST import HaloWST

I cataloghi di aloni sono elenchi degli aloni presi come punti massivi

Ognuno ha una propria massa e un numero di particelle di CDM da cui è formato

In [2]:
# input file
# need to select a folder and there must be
# a folder named '/groups_00*snapnum*'
snapdir = '/home/fuffolo97/TESI/Try_02-HaloCatalogue/Halos/fiducial/1'
snapnum = 2

# determine the reìdshift of the catalogue
z_dict = {4:0.0, 3:0.5, 2:1.0, 1:2.0, 0:3.0}
redshift = z_dict[snapnum]

In [3]:
# read the halo catalogue
datas = readfof.FoF_catalog(snapdir, snapnum, read_IDs=False)

In [4]:
# how to get the propeties of the halos
pos_h = datas.GroupPos/1e3              # positions in Mpc/h
mass = datas.GroupMass * 1e10           # masses in M_sun/h
# vel_h = datas.GroupVel * (1.0*redshift) # peculiar velocities in km/s
# Npart = datas.GroupLen                  # number of CDM particles in the halo

#### Create density matrix from Quijote Halo simulation

In [6]:
import sys

sys.path.insert(1, '../MyFunc')
from myCIC import cic, miafunzione

In [6]:
# import cProfile
# cProfile.run('dens = cic(pos_h, mass, 128, 1000)', "results.prof")

#dens = cic_multiT(pos_h, mass, 128)

CIC non multithread:
- Before for loops:  0.008156299591064453 s 
- For loops:  135.50461220741272 s
- TOT $\sim$ 2m 15s

CIC non multithread (inverted for loops): same

CIC using function inside 'for i in ...':
- TOT $\sim$ 2m 11s

CIC movingcheck of boundaries before 'for x in range...'
- 1m 15s

CIC calculating before 'forx in range...' CELL[0], CELL[1],...:
- 1m 7s

In [7]:
dens2 = cic(pos_h, mass, 128)

In [8]:
# %load_ext snakeviz

time: 3m 7.1s

#### Calculate scattering transform coefficients

In [9]:
# parameters for HarmonicScattering3D
M, N, O = 128, 128, 128
J, L = 4, 4

In [10]:
S = HarmonicScattering3D(J=J, L=L, shape=(M, N, O), sigma_0=0.8, integral_powers=[0.8])

In [11]:
# argument of "torch.from_numpy(...)" must be numpy.float32()
Sx = S.scattering(torch.from_numpy(dens2))

In [12]:
first_order = []
second_order = []

for i in range(L+1):
    for j in range(J+1):
        first_order.append(Sx[j, i, 0])
    second_order.append(Sx[5:,i,0].numpy())

In [13]:
import dis

dis.dis(miafunzione1)

193           0 LOAD_GLOBAL              0 (np)
              2 LOAD_METHOD              1 (zeros)
              4 LOAD_CONST               1 ((2, 2, 2))
              6 CALL_METHOD              1
              8 STORE_FAST               7 (cube)

194          10 LOAD_FAST                6 (vecs)
             12 GET_ITER
        >>   14 FOR_ITER                34 (to 84)
             16 STORE_FAST               8 (v)

195          18 LOAD_FAST                7 (cube)
             20 LOAD_FAST                8 (v)
             22 LOAD_CONST               2 (0)
             24 BINARY_SUBSCR
             26 BINARY_SUBSCR
             28 LOAD_FAST                8 (v)
             30 LOAD_CONST               3 (1)
             32 BINARY_SUBSCR
             34 BINARY_SUBSCR
             36 LOAD_FAST                8 (v)
             38 LOAD_CONST               4 (2)
             40 BINARY_SUBSCR
             42 DUP_TOP_TWO
             44 BINARY_SUBSCR
             46 LOAD_FAST             

All the previous calcs can be done now simply by:

```
$ import sys
$ sys.path.insert(1, '/home/fuffolo97/TESI/MyFunc')
$ from CalcWST import HaloWST
$ snapdir = '/path/to/files/without/groups_'
$ WST = HaloWST(snapdir)
```